In [ ]:
# Jorge Barcia Belinchón 100496595 partner Paloma Núñez Guerrero

In [ ]:
import numpy as np
import multiprocessing as mp
import time

In [ ]:
# Function of numpy we have used
#    1. To create matrix: np.random.rand()
#    2. Verify if the matrix if invertible:
#           numpy.linalg.det() (det 0 is not invertible)
#    3. Verify result: numpy.allclose()
#    4. Multiplication: numpy.dot()
#    5. Manipulation: numpy.transpose(), numpy.reshape(), etc.

# Create initial matrix

Variables to change:
* numN: number of rows-columns of the matrix
* min_value: min value in the matrix
* max_value: max value in the matrix

In [ ]:
# First of all, create the matrix
def create_matrix(numN, min_value, max_value, int_matrix=False):
    matrix_good = False
    matrix = np.zeros((numN, numN))
    while not matrix_good:
        # Generate matrix
        if int_matrix:
            matrix = np.random.randint(low=min_value, high=max_value, size=(numN, numN))
        else:
            matrix = np.random.rand(numN, numN) * (max_value - min_value) + min_value

        # Check if it's invertible
        determinant = np.linalg.det(matrix)
        if determinant != 0 and not np.isinf(determinant) and not np.isnan(determinant):
            matrix_good = True

    return matrix

In [ ]:
# Set the num cores
NUMREPORTEDCORES=mp.cpu_count()
#Should be the number of real computational cores
NUMCORES=NUMREPORTEDCORES

# Practical Work 1

In [ ]:
def init_sharedarray(shared_array, matrix_shape, minors_array, cofactor_array):
    #global shared_space
    global shared_matrix
    global matrix_lock
    global shared_minors_matrix
    global shared_cofactor_matrix

    shared_matrix = np.frombuffer(shared_array.get_obj(), dtype=np.float64).reshape(matrix_shape)
    shared_minors_matrix = np.frombuffer(minors_array.get_obj(), dtype=np.float64).reshape(matrix_shape)
    shared_cofactor_matrix = np.frombuffer(cofactor_array.get_obj(), dtype=np.float64).reshape(matrix_shape)
    matrix_lock = mp.Lock()


class InversedMatrix:

  def __init__(self, matrix):
      self.matrix = matrix
      self.num_rows, self.num_cols = matrix.shape
      self.shared_array = mp.Array('d', self.num_rows * self.num_cols)
      self.shared_minors_matrix_space = mp.Array('d', self.num_rows * self.num_cols)
      self.shared_cofactor_matrix_space = mp.Array('d', self.num_rows * self.num_cols)

      self.copy_matrix_to_shared()

  def copy_matrix_to_shared(self):
      np.copyto(np.frombuffer(self.shared_array.get_obj(), dtype=np.float64).reshape((self.num_rows, self.num_cols)), self.matrix.astype(np.float64))

  # ------------ STEP 1 ------------
  @staticmethod
  def calculate_determinant_minor(matrix, row, column):
      # Eliminate row and column indicated
      submatrix = np.delete(np.delete(matrix, row, axis=0), column, axis=1)
      return np.linalg.det(submatrix)

  @staticmethod
  def compute_minor(row):
      global shared_matrix
      global shared_minors_matrix
      global matrix_lock

      num_cols = shared_matrix.shape[1]
      local_minors_row = np.zeros(num_cols)

      for column in range(num_cols):
          local_minors_row[column] = InversedMatrix.calculate_determinant_minor(shared_matrix, row, column)


      with matrix_lock:
          shared_minors_matrix[row, :] = local_minors_row

  # Calculate minor matrix
  def calculate_minors_matrix(self):
      rows = range(self.num_rows)

      with mp.Pool(processes=NUMCORES, initializer=init_sharedarray,
                     initargs=[self.shared_array, (self.num_rows, self.num_cols), self.shared_minors_matrix_space, self.shared_cofactor_matrix_space]) as pool:
            pool.map(self.compute_minor, rows)

      # Convert to numpy array
      minors_matrix = np.frombuffer(self.shared_minors_matrix_space.get_obj(), dtype=np.float64).reshape((self.num_rows, self.num_cols))

      return minors_matrix


  # ------------ STEP 2 ------------
  # Not neccesary to do paralellization, but the method is more optimal, which is what we seek.
  """@staticmethod
  def calculate_cofactor_matrix(minors_matrix):
      sign_matrix = np.fromfunction(lambda i, j: (-1) ** (i + j), minors_matrix.shape, dtype=int)

      cofactor_matrix = minors_matrix * sign_matrix

      return cofactor_matrix"""

  @staticmethod
  def compute_cofactor_element(row_col):
      global shared_minors_matrix
      global shared_cofactor_matrix
      global matrix_lock

      row, col = row_col
      sign = (-1) ** (row + col)

      cofactor_value = sign * shared_minors_matrix[row, col]

      with matrix_lock:
          shared_cofactor_matrix[row, col] = cofactor_value

  def calculate_cofactor_matrix(self):
      indices = [(row, col) for row in range(self.num_rows) for col in range(self.num_cols)]

      with mp.Pool(processes=NUMCORES, initializer=init_sharedarray,
                    initargs=[self.shared_array, (self.num_rows, self.num_cols), self.shared_minors_matrix_space, self.shared_cofactor_matrix_space]) as pool:
          pool.map(InversedMatrix.compute_cofactor_element, indices)

      cofactor_matrix = np.frombuffer(self.shared_cofactor_matrix_space.get_obj(), dtype=np.float64).reshape((self.num_rows, self.num_cols))
      return cofactor_matrix


 # ------------ STEP 3 ------------
  @staticmethod
  def calculate_adjugate(cofactor_matrix):
    return np.transpose(cofactor_matrix)


 # ------------ STEP 4 ------------
def calculate_inverse(matrix_A):
    inverse = InversedMatrix(matrix_A)

    determinant = np.linalg.det(matrix_A)

    minors_matrix = inverse.calculate_minors_matrix()
    cofactor_matrix = inverse.calculate_cofactor_matrix()
    adjugate_matrix = inverse.calculate_adjugate(cofactor_matrix)

    """print("Original matrix:")
    print(matrix_A)
    print("\nMinors matrix:")
    print(minors_matrix)
    print("\nCofactor matrix:")
    print(cofactor_matrix)
    print("\nMAdjugate matrix:")
    print(adjugate_matrix)"""

    return adjugate_matrix / determinant

# ------------ EVALUATION ------------
def check_inverse(original_matrix, inverse_matrix):
    identity_matrix = np.eye(original_matrix.shape[0])

    product = np.dot(original_matrix, inverse_matrix)

    return np.allclose(product, identity_matrix, atol=1e-10, rtol=1e-10)

def performance_test(numN, min_value, max_value, iterations=10):
    total_time = 0
    for _ in range(iterations):
        matrix_A = create_matrix(numN, min_value, max_value)

        start_time = time.time()
        inverse_matrix = calculate_inverse(matrix_A)
        end_time = time.time()

        is_correct = check_inverse(matrix_A, inverse_matrix)
        print("\n¿The inverse is correct?", is_correct)

        total_time += (end_time - start_time)

    average_time = total_time / iterations
    print(f"Tamaño de la matriz: {numN}x{numN}, tiempo promedio para calcular la inversa: {average_time:.4f} segundos.")


In [ ]:
if __name__ == "__main__":
    np.random.seed(42)
    numN=500
    min_value = -0.25 #-1000
    max_value = 0.25 # 1000

    # for one: iterations = 1
    
    performance_test(numN, min_value, max_value, iterations=1)


# Conclusion: 
 > The work consisted of four parts, the most complicated of which was the first. 
At first, we did not manage to get the values stored in the minors array, so the variable was introduced in the init_sharedarray() function. After that, we faced running problems, and to solve this we introduced the variable in the init_sharedarray() function.
---
> After that, we were confronted with race problems, and locks were introduced to solve them. Finally, to finish the first step, attention was paid to memory accesses, trying to reduce them as much as possible.
Moving on to the second step, although it is not necessary to parallelise it, it was done to reduce memory accesses, so that when large arrays are used, it is as fast as possible. Similarly, the function that does not contain parallelisation has been kept commented out.
---
> To execute the whole loop, the main must be executed, so that the number of iterations, the size of the matrix, and the range of values it must contain are established.

# Conclusion
 > For this work we have follow a by parts aproach, being the first one the hardest as it required some majors changes of our initial aproach especificaly in storing values in the minors array and some running problems, this problems were solve by introducing them in the init_sharedarray() function.
> the next step was solving with the race problem that was fairly discus truh out the course therefore using locks was a straigh forwar solution. After implementing locks we needed to implement the memory access in the smartes way possible to not consume to many resources.
>The next part was to parallelise the proces to improve performance and reduce memory access.
---
